# Population Statistics

In [11]:
### Imports

In [11]:
%pip install wbdata
%pip install pandas

import wbdata
import pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
## Topic Selection

In [75]:
# Returns the number of displaced people given the country/region
# This allows us to access any listed country/region within wbdata and see the number of people displaced between certain years
# **IMPORTANT: Some countries didn't track certain years, so a timeframe that matches most countries is required (eg. 2008-2017)**

population_displaced = wbdata.get_indicator(source=2) # World Development Indicators
variable = {"VC.IDP.NWDS": "Internally displaced persons, new displacement associated with disasters (number of cases)"} #SOURCE

dataframe = wbdata.get_dataframe(variable) # Creates dataframe
cleaned_df = dataframe.dropna() # Drops NaN values

df_index = cleaned_df.reset_index() # DF is in multi-indexed format; this resets the index to be more easily accessed
filtered_df = df_index[df_index['country'] == "World"] # Select location
 
filtered_df.head(50) #First 50 displayed; overshoot to be safe

,country,date,"Internally displaced persons, new displacement associated with disasters (number of cases)"
541,World,2022,32533018.0
542,World,2021,23725311.0
543,World,2020,30984299.0
544,World,2019,25285518.0
545,World,2018,17462844.0
546,World,2017,18587056.0
547,World,2016,24418613.0
548,World,2015,19188817.0
549,World,2014,19119870.0
550,World,2013,22125771.0


In [94]:
# Returns the population size of a given country
# SOURCE = 40; Population estimates and projections

world_population = {"SP.POP.TOTL":"World Population"}
female_population = {"SP.POP.TOTL.FE.IN": "Population, female"}
male_population = {"SP.POP.TOTL.MA.IN":"Population, male"}

total_count = wbdata.get_dataframe(world_population, country="WLD") #"WLD" will be replaced with the given country ID as a parameter
male_count = wbdata.get_dataframe(male_population, country="WLD")
female_count = wbdata.get_dataframe(female_population, country="WLD")

,World Population
date,
2022,7.950947e+09
2021,7.888306e+09
2020,7.820206e+09
2019,7.741775e+09
2018,7.660371e+09
...,...
1964,3.260442e+09
1963,3.193429e+09
1962,3.126850e+09


In [ ]:
def population(year, sex, age_range, location):

    pass